In [6]:
import os 
import numpy as np 
import pandas as pd 
from sklearn.model_selection   import train_test_split

In [2]:
# pip install numpy


In [3]:
#pip install -U scikit-learn


In [4]:
#pip install pandas


## Reading data 

In [12]:
project_directory= r"C:\Users\USER\Desktop\sagemaker2"
data_directory="data"

In [16]:
os.path.join(project_directory,data_directory,"flight_price.csv")

'C:\\Users\\USER\\Desktop\\sagemaker2\\data\\flight_price.csv'

In [17]:
def get_data(name):
    file_name = f"{name}.csv"
    file_path = os.path.join(project_directory,data_directory,file_name)
    return pd.read_csv (file_path)

In [20]:
flights= get_data("flight_price")
flights.head(3)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882


In [9]:
#pd.read_csv("flight_price.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'flight_price.csv'

In [31]:
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


# Prilinary Checks 
## checking data types

In [33]:
flights.dtypes

Airline            object
Date_of_Journey    object
Source             object
Destination        object
Route              object
Dep_Time           object
Arrival_Time       object
Duration           object
Total_Stops        object
Additional_Info    object
Price               int64
dtype: object

#### obesrvation 
- (Date_of_Journey ,Dep_Time  and  Arrival_Time)    object   we have to change it to Date_time
      
- Duration    type is mixed in hrs and minutes we need to make it in one measurable unit
- Total_Stops   is object we need to change it to numeric        
 

#### checking duplicates 

In [34]:
flights.duplicated().sum()

220

In [37]:
(flights
 .loc[flights.duplicated(keep =False)]
.sort_values(["Airline", "Date_of_Journey", "Source", "Destination"]))

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6321,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
9848,Air India,01/03/2019,Banglore,New Delhi,BLR → BOM → AMD → DEL,08:50,23:55 02 Mar,39h 5m,2 stops,No info,17135
572,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
8168,Air India,03/03/2019,Banglore,New Delhi,BLR → DEL,21:10,23:55,2h 45m,non-stop,No info,7591
1495,Air India,1/04/2019,Kolkata,Banglore,CCU → DEL → COK → BLR,10:00,01:20 02 Apr,15h 20m,2 stops,No info,10408
...,...,...,...,...,...,...,...,...,...,...,...
2692,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
2870,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,05:45,08:35,2h 50m,non-stop,No check-in baggage included,4273
3711,SpiceJet,24/03/2019,Banglore,New Delhi,BLR → DEL,20:30,23:20,2h 50m,non-stop,No check-in baggage included,3873
2634,Vistara,24/03/2019,Banglore,New Delhi,BLR → DEL,11:30,14:10,2h 40m,non-stop,No info,5403


### duplicates must be removed 

## Detailed Analysis

### Airline 

In [43]:
flights.Airline

0             IndiGo
1          Air India
2        Jet Airways
3             IndiGo
4             IndiGo
            ...     
10678       Air Asia
10679      Air India
10680    Jet Airways
10681        Vistara
10682      Air India
Name: Airline, Length: 10683, dtype: object

In [45]:
flights.Airline.unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

- Vistara Premium  economy and Jet airways  Bussiness need to Replaces with vistara and Jet Airways 

In [51]:
(
    flights
    .Airline
    .str.replace(" Premium economy","")
    .str.replace(" Business","")
    .str.title()
    .unique()
    
)

array(['Indigo', 'Air India', 'Jet Airways', 'Spicejet',
       'Multiple Carriers', 'Goair', 'Vistara', 'Air Asia', 'Trujet'],
      dtype=object)

###  Date_of_Journey

In [52]:
flights.Date_of_Journey

0        24/03/2019
1         1/05/2019
2         9/06/2019
3        12/05/2019
4        01/03/2019
            ...    
10678     9/04/2019
10679    27/04/2019
10680    27/04/2019
10681    01/03/2019
10682     9/05/2019
Name: Date_of_Journey, Length: 10683, dtype: object

In [53]:
pd.to_datetime(flights.Date_of_Journey,dayfirst=True)

0       2019-03-24
1       2019-05-01
2       2019-06-09
3       2019-05-12
4       2019-03-01
           ...    
10678   2019-04-09
10679   2019-04-27
10680   2019-04-27
10681   2019-03-01
10682   2019-05-09
Name: Date_of_Journey, Length: 10683, dtype: datetime64[ns]

### Source

In [62]:
flights.Source.unique()

array(['Banglore', 'Kolkata', 'Delhi', 'Chennai', 'Mumbai'], dtype=object)

### Destination

In [63]:
flights.Destination.unique()

array(['New Delhi', 'Banglore', 'Cochin', 'Kolkata', 'Delhi', 'Hyderabad'],
      dtype=object)

- both columns looks alright
- Also i m going to remove the ROUTE column 

### Dep_time

In [66]:
flights.Dep_Time

0        22:20
1        05:50
2        09:25
3        18:05
4        16:50
         ...  
10678    19:55
10679    20:45
10680    08:20
10681    11:30
10682    10:55
Name: Dep_Time, Length: 10683, dtype: object

In [67]:
# dtyoe object  so we can operatye through  string operation

In [68]:
(
    flights
    .Dep_Time
    .loc[lambda ser: ser.str.contains("[^0-9:]")]
)

Series([], Name: Dep_Time, dtype: object)

In [72]:
pd.to_datetime(flights.Dep_Time).dt.time

C:\Users\USER\AppData\Local\Temp\ipykernel_11688\1056054517.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(flights.Dep_Time).dt.time


0        22:20:00
1        05:50:00
2        09:25:00
3        18:05:00
4        16:50:00
           ...   
10678    19:55:00
10679    20:45:00
10680    08:20:00
10681    11:30:00
10682    10:55:00
Name: Dep_Time, Length: 10683, dtype: object

### Arrival_Time 

In [76]:
flights.Arrival_Time

0        01:10 22 Mar
1               13:15
2        04:25 10 Jun
3               23:30
4               21:35
             ...     
10678           22:25
10679           23:20
10680           11:20
10681           14:10
10682           19:15
Name: Arrival_Time, Length: 10683, dtype: object

In [79]:
(
    flights
    .Arrival_Time
    .loc[lambda ser : ser.str.contains("[^0-9:]")]
    .str.split(" " , n = 1 )
    .str.get(0)
    .unique()
)

array(['01:10', '04:25', '10:25', '05:05', '09:20', '19:15', '12:35',
       '13:20', '08:15', '03:35', '19:00', '11:05', '10:55', '00:15',
       '02:45', '12:00', '01:30', '16:45', '09:25', '04:40', '11:25',
       '14:25', '14:35', '09:05', '01:35', '10:05', '11:10', '07:45',
       '00:55', '23:35', '00:40', '18:50', '07:55', '18:15', '13:15',
       '09:45', '06:50', '15:15', '07:40', '02:20', '18:30', '19:45',
       '19:50', '01:00', '00:05', '16:20', '00:30', '00:50', '13:45',
       '09:15', '07:10', '08:55', '04:45', '05:35', '09:00', '10:45',
       '22:10', '22:40', '05:25', '02:15', '20:20', '06:35', '01:20',
       '19:10', '10:50', '23:25', '12:15', '10:40', '23:15', '22:55',
       '09:10', '18:10', '00:25', '20:45', '21:05', '23:00', '00:45',
       '04:35', '18:00', '12:30', '02:10', '00:20', '23:20', '12:25',
       '22:35', '20:25', '00:35', '16:50', '09:30', '12:05', '10:35',
       '15:05', '18:40', '09:35', '08:50', '16:40', '19:40', '16:10',
       '21:20', '07:

### Duration

In [83]:
flights.Duration  #  need to change it to one si unit im chnaging it to minutes

0        2h 50m
1        7h 25m
2           19h
3        5h 25m
4        4h 45m
          ...  
10678    2h 30m
10679    2h 35m
10680        3h
10681    2h 40m
10682    8h 20m
Name: Duration, Length: 10683, dtype: object

(
    flights
    .Duration
    .loc[lambda ser : ~ser.str.contains("h")]
    #.unique()
)

In [91]:
""" invalid duration type   5 minutes is not possible at all from Mumbai to Hydrabad """
flights.iloc[[6474]] 


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
6474,Air India,6/03/2019,Mumbai,Hyderabad,BOM → GOI → PNQ → HYD,16:50,16:55,5m,2 stops,No info,17327


In [106]:
(
    flights
    .Duration
    .drop(index=6474)
    .str.split(" ",expand =True)
    .set_axis(["hour","minute"],axis =1)
    .assign(
        hour =lambda df_ :(
            df_
            .hour
            .str.replace("h","")
            .astype(int)
            .mul(60)
        ),
        minute=lambda df_:(
            df_
            .minute
            .str.replace("m","")
            .fillna("0")
            .astype(int)
        )
        
    )
    #.dtypes
    #.isna().sum()
    .sum(axis=1)
    .rename("duration_in_minutes")
    .to_frame()
    .join(flights.Duration.drop(index=6474))
)

,duration_in_minutes,Duration
0,170,2h 50m
1,445,7h 25m
2,1140,19h
3,325,5h 25m
4,285,4h 45m
...,...,...
10678,150,2h 30m
10679,155,2h 35m
10680,180,3h
10681,160,2h 40m


# Total stops

In [109]:
flights.Total_Stops.unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [114]:
(
    flights
    .Total_Stops
    .replace("non-stop", "0")
    .str.replace(" stops?","",regex=True)# (" stop")  or (" stops") 
    .pipe(lambda ser : pd.to_numeric(ser))
    
    
)

0        0.0
1        2.0
2        2.0
3        1.0
4        1.0
        ... 
10678    0.0
10679    0.0
10680    0.0
10681    0.0
10682    2.0
Name: Total_Stops, Length: 10683, dtype: float64

## Additional info 

In [118]:
flights.Additional_Info.unique() # spelling mistake No info  No Info

array(['No info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover', 'No Info',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

# cleaning data

In [107]:
def convert_to_minute(ser):
    return (
        ser
        .str.split(" ",expand =True)
        .set_axis(["hour","minute"],axis =1)
        .assign(
            hour =lambda df_ :(
                df_
                .hour
                .str.replace("h","")
                .astype(int)
                .mul(60)
            ),
            minute=lambda df_:(
                df_
                .minute
                .str.replace("m","")
                .fillna("0")
                .astype(int)
            )
            
        )
        #.dtypes
        #.isna().sum()
        .sum(axis=1)
        
    )

In [126]:
def clean_data(df):
    return (
        df
        .drop(index=[6474])
        .drop_duplicates()
        .assign(**{
            col:df[col].str.strip()
            for col in df.select_dtypes(include="O").columns
        })
        .rename(columns=str.lower)
        .assign(
            airline= lambda df_:(
                    df_
                    .airline
                    .str.replace(" Premium economy","")
                    .str.replace(" Business","")
                    .str.title()
            ),
            date_of_journey = lambda df_:pd.to_datetime(df_.date_of_journey,dayfirst=True),
            dep_time =lambda df_:pd.to_datetime(df_.dep_time).dt.time,
            arrival_time =lambda df_:pd.to_datetime(df_.arrival_time).dt.time,
            duration =lambda df_ :df_.duration.pipe(convert_to_minute), 
            # also 1 way duration =lambda df_ : convert_to_minute(df_.duration)'
            total_stops =lambda df_:(
                df_
                .total_stops
                .replace("non-stop", "0")
                .str.replace(" stops?","",regex=True)
                .pipe(lambda ser : pd.to_numeric(ser))
                 ),
             additional_info =lambda df_ : df_.additional_info.replace("No info","No Info")   
        )
        .drop(columns ="route") # droping route column
    )

In [127]:
clean_data(flights).additional_info.unique()

C:\Users\USER\AppData\Local\Temp\ipykernel_11688\1735572152.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time =lambda df_:pd.to_datetime(df_.dep_time).dt.time,
C:\Users\USER\AppData\Local\Temp\ipykernel_11688\1735572152.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time =lambda df_:pd.to_datetime(df_.arrival_time).dt.time,


array(['No Info', 'In-flight meal not included',
       'No check-in baggage included', '1 Short layover',
       '1 Long layover', 'Change airports', 'Business class',
       'Red-eye flight', '2 Long layover'], dtype=object)

In [129]:
flights_cleaned = clean_data(flights)
flights_cleaned

C:\Users\USER\AppData\Local\Temp\ipykernel_11688\1735572152.py:20: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dep_time =lambda df_:pd.to_datetime(df_.dep_time).dt.time,
C:\Users\USER\AppData\Local\Temp\ipykernel_11688\1735572152.py:21: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  arrival_time =lambda df_:pd.to_datetime(df_.arrival_time).dt.time,


,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Indigo,2019-03-24,Banglore,New Delhi,22:20:00,01:10:00,170,0.0,No Info,3897
1,Air India,2019-05-01,Kolkata,Banglore,05:50:00,13:15:00,445,2.0,No Info,7662
2,Jet Airways,2019-06-09,Delhi,Cochin,09:25:00,04:25:00,1140,2.0,No Info,13882
3,Indigo,2019-05-12,Kolkata,Banglore,18:05:00,23:30:00,325,1.0,No Info,6218
4,Indigo,2019-03-01,Banglore,New Delhi,16:50:00,21:35:00,285,1.0,No Info,13302
...,...,...,...,...,...,...,...,...,...,...
10678,Air Asia,2019-04-09,Kolkata,Banglore,19:55:00,22:25:00,150,0.0,No Info,4107
10679,Air India,2019-04-27,Kolkata,Banglore,20:45:00,23:20:00,155,0.0,No Info,4145
10680,Jet Airways,2019-04-27,Banglore,Delhi,08:20:00,11:20:00,180,0.0,No Info,7229
10681,Vistara,2019-03-01,Banglore,New Delhi,11:30:00,14:10:00,160,0.0,No Info,12648


## Split the Data

In [130]:
flights_final= flights_cleaned.sample(1000)

In [131]:
x=flights_final.drop(columns="price")
y=flights_final.price.copy()

In [133]:
x_,x_test,y_,y_test=train_test_split(x,y,test_size=0.2,random_state=42 )

x_train,x_val,y_train,y_val = train_test_split(x_,y_,test_size=0.2,random_state=42)

print(x_train.shape,y_train.shape)
print(x_val.shape,y_val.shape)
print(x_test.shape,y_test.shape)

(640, 9) (640,)
(160, 9) (160,)
(200, 9) (200,)


## Export the subsets

In [134]:
def export_data(x,y,name):
    file_name=f"{name}.csv"
    file_path =os.path.join(project_directory,data_directory,file_name)
    
    x.join(y).to_csv(file_path,index=False)
    
    return pd.read_csv(file_path).head()

In [135]:
export_data(x_train,y_train,"train")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-03-24,Chennai,Kolkata,11:40:00,13:55:00,135,0.0,No Info,4405
1,Indigo,2019-06-27,Banglore,Delhi,22:10:00,01:00:00,170,0.0,No Info,3419
2,Indigo,2019-05-18,Kolkata,Banglore,08:10:00,13:00:00,290,1.0,No Info,5279
3,Indigo,2019-06-06,Delhi,Cochin,06:50:00,22:30:00,940,1.0,No Info,6674
4,Jet Airways,2019-05-27,Delhi,Cochin,11:30:00,12:35:00,1505,1.0,In-flight meal not included,12898


In [136]:
export_data(x_test,y_test,"test")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-05-21,Kolkata,Banglore,07:00:00,20:25:00,805,1.0,No Info,11096
1,Jet Airways,2019-03-09,Delhi,Cochin,06:50:00,04:25:00,1295,1.0,No Info,16289
2,Air India,2019-03-12,Mumbai,Hyderabad,21:05:00,22:25:00,80,0.0,No Info,4307
3,Spicejet,2019-03-03,Kolkata,Banglore,17:15:00,19:45:00,150,0.0,No Info,6251
4,Indigo,2019-03-27,Delhi,Cochin,10:45:00,21:00:00,615,1.0,No Info,6838


In [137]:
export_data(x_val,y_val,"val")

,airline,date_of_journey,source,destination,dep_time,arrival_time,duration,total_stops,additional_info,price
0,Air India,2019-04-01,Kolkata,Banglore,20:30:00,12:00:00,930,1.0,No Info,7893
1,Spicejet,2019-06-06,Kolkata,Banglore,15:55:00,21:25:00,330,1.0,No Info,8224
2,Goair,2019-04-01,Banglore,Delhi,07:45:00,10:40:00,175,0.0,No Info,4239
3,Indigo,2019-04-03,Mumbai,Hyderabad,16:55:00,18:25:00,90,0.0,No Info,3175
4,Spicejet,2019-04-01,Kolkata,Banglore,15:55:00,21:25:00,330,1.0,No Info,7646
